# 5. Importance Sampling for Neural Network Training (FIXED)

In this notebook, we explore **importance sampling** as a technique for improving neural network inference on extreme parameter values.

## Key Fixes Applied:
1. **Numerical stability** in AR(1) simulation (clamp rho, add epsilon)
2. **No transformations during training** (train on raw values)
3. **Conservative weight clamping** [0.1, 10.0] to prevent extreme normalized weights
4. **Extensive NaN/Inf checking** with detailed error messages
5. **Removed transformations from evaluation** (fixed sigma offset issue)
6. **Added Gibbs sampler comparison**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy import stats
import pandas as pd

np.random.seed(42)
torch.manual_seed(42)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
A = 5.0  # Upper bound for sigma
T = 64   # Time series length

def simulate_ar1_batch(rho, sigma, T, batch_size, device='cpu'):
    """Vectorized AR(1) simulation for a batch - WITH NUMERICAL STABILITY"""
    eps = torch.randn(batch_size, T, device=device) * sigma[:, None]
    
    # CRITICAL FIX: Clamp rho and add epsilon to prevent overflow
    rho_safe = torch.clamp(rho, -0.9999, 0.9999)
    x0 = torch.randn(batch_size, device=device) * sigma / torch.sqrt(1 - rho_safe**2 + 1e-8)
    
    x = torch.zeros(batch_size, T, device=device)
    x[:, 0] = rho * x0 + eps[:, 0]
    
    for t in range(1, T):
        x[:, t] = rho * x[:, t-1] + eps[:, t]
    
    return x

def sample_from_proposal(batch_size, alpha_rho=0.5, alpha_sigma=0.5, beta_sigma=0.5, A=5.0, device='cpu'):
    """Sample (rho, sigma) from Beta-based proposal"""
    rho_tilde = np.random.beta(alpha_rho, alpha_rho, size=batch_size)
    sigma_tilde = np.random.beta(alpha_sigma, beta_sigma, size=batch_size)
    
    rho = 2 * rho_tilde - 1
    sigma = A * sigma_tilde
    
    rho = torch.tensor(rho, dtype=torch.float32, device=device)
    sigma = torch.tensor(sigma, dtype=torch.float32, device=device)
    
    return rho, sigma

def compute_importance_weights(rho, sigma, alpha_rho=0.5, alpha_sigma=0.5, beta_sigma=0.5, A=5.0):
    """Compute importance weights w = p(theta) / q(theta)"""
    if torch.is_tensor(rho):
        rho = rho.cpu().numpy()
        sigma = sigma.cpu().numpy()
    
    rho_tilde = (rho + 1) / 2
    sigma_tilde = sigma / A
    
    q_rho = stats.beta.pdf(rho_tilde, alpha_rho, alpha_rho) / 2
    q_sigma = stats.beta.pdf(sigma_tilde, alpha_sigma, beta_sigma) / A
    
    q_theta = q_rho * q_sigma
    p_theta = 1.0 / (2 * A)
    
    weights = p_theta / q_theta
    
    return weights

print("Setup functions loaded")

In [ ]:
class GRUPosteriorEstimator(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=2, mlp_hidden=64):
        super().__init__()
        
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        
        self.estimator = nn.Sequential(
            nn.Linear(hidden_dim, mlp_hidden),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(mlp_hidden, mlp_hidden),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(mlp_hidden, output_dim)
        )
    
    def forward(self, x):
        x = x.unsqueeze(-1)
        gru_out, h_n = self.gru(x)
        last_hidden = h_n[-1]
        return self.estimator(last_hidden)

print("✓ GRU model defined")

In [ ]:
def train_standard(model, n_epochs=100, batch_size=128, lr=0.001, device='cpu', A=5.0, T=64):
    """Standard training - NO TRANSFORMATIONS during training"""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
    loss_fn = nn.MSELoss()
    
    train_losses = []
    model.train()
    
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        n_batches = 128
        
        for _ in range(n_batches):
            rho = torch.empty(batch_size, device=device).uniform_(-1, 1)
            sigma = torch.empty(batch_size, device=device).uniform_(0, A)
            x = simulate_ar1_batch(rho, sigma, T, batch_size, device=device)
            
            theta_pred = model(x)
            loss = loss_fn(theta_pred, torch.stack([rho, sigma], axis=1))
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / n_batches
        train_losses.append(avg_loss)
        scheduler.step(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.6f}")
    
    return train_losses

def train_importance_sampling(model, n_epochs=100, batch_size=128, lr=0.001, 
                              alpha_rho=0.5, alpha_sigma=0.5, beta_sigma=0.5,
                              device='cpu', A=5.0, T=64):
    """Training with importance sampling - FIXED"""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
    
    train_losses = []
    model.train()
    
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        n_batches = 128
        
        for batch_idx in range(n_batches):
            rho, sigma = sample_from_proposal(batch_size, alpha_rho, alpha_sigma, beta_sigma, A, device)
            
            weights = compute_importance_weights(rho, sigma, alpha_rho, alpha_sigma, beta_sigma, A)
            weights = torch.tensor(weights, dtype=torch.float32, device=device)
            weights = torch.nan_to_num(weights, nan=1.0, posinf=10.0, neginf=0.1)
            
            # CRITICAL FIX: Conservative clamping [0.1, 10.0]
            weights = torch.clamp(weights, min=0.1, max=10.0)
            weights = weights / weights.sum()
            
            x = simulate_ar1_batch(rho, sigma, T, batch_size, device=device)
            
            if torch.isnan(x).any() or torch.isinf(x).any():
                continue
            
            theta_pred = model(x)
            
            if torch.isnan(theta_pred).any() or torch.isinf(theta_pred).any():
                continue
            
            loss_per_sample = ((theta_pred - torch.stack([rho, sigma], dim=1))**2).sum(dim=1)
            loss = torch.sum(weights * loss_per_sample)
            
            if torch.isnan(loss) or torch.isinf(loss):
                continue
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / n_batches
        train_losses.append(avg_loss)
        scheduler.step(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            ess = 1.0 / torch.sum(weights**2)
            print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.6f}, ESS: {ess.item():.1f}/{batch_size}")
    
    return train_losses

print("✓ Training functions defined")

## Training Models

In [ ]:
# Train standard model
print("Training STANDARD model (uniform sampling)...")
print("="*60)
model_standard = GRUPosteriorEstimator(hidden_dim=64, num_layers=2, mlp_hidden=64).to(device)

losses_standard = train_standard(
    model_standard, n_epochs=100, batch_size=128, lr=0.001,
    device=device, A=A, T=T
)

print("\n" + "="*60)
print("Training IMPORTANCE SAMPLING model with Beta(0.7, 0.7)...")
print("="*60)
model_is = GRUPosteriorEstimator(hidden_dim=64, num_layers=2, mlp_hidden=64).to(device)

losses_is = train_importance_sampling(
    model_is, n_epochs=100, batch_size=128, lr=0.001,
    alpha_rho=0.7, alpha_sigma=0.7, beta_sigma=0.7,
    device=device, A=A, T=T
)

In [ ]:
# Plot training curves
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(losses_standard, label='Standard (uniform)', linewidth=2)
plt.plot(losses_is, label='Importance Sampling (Beta 0.7)', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

plt.subplot(1, 2, 2)
plt.plot(losses_standard[10:], label='Standard', linewidth=2)
plt.plot(losses_is[10:], label='IS (Beta 0.7)', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss (after epoch 10)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Evaluation on Test Sets

## Gibbs Sampler for Comparison

In [ ]:
def truncated_normal(mu, sigma, a=-1, b=1):
    """Sample from truncated normal distribution"""
    while True:
        sample = np.random.normal(mu, sigma)
        if a <= sample <= b:
            return sample

def truncated_gamma(shape, scale, a=1/A**2):
    """Sample from truncated gamma distribution"""
    while True:
        sample = np.random.gamma(shape, scale)
        if sample >= a:
            return sample

def gibbs(x, n_iter=1000):
    """Gibbs sampler for AR(1) parameters"""
    T_local = len(x) - 1
    
    Q = np.sum(x[:-1]**2)
    rho_hat = np.sum(x[:-1] * x[1:]) / Q
    const = np.sum(x[1:]**2) - Q * rho_hat**2
    
    S = lambda rho: Q * (rho - rho_hat)**2 + const
    
    samples = np.zeros((n_iter, 2))
    for i in range(n_iter):
        # Sample lambda (precision)
        lambda_ = truncated_gamma(T_local/2, 2/(S(rho_hat) + 1e-6))
        sigma = 1/np.sqrt(lambda_)
        samples[i, 1] = sigma
        
        # Sample rho
        samples[i, 0] = truncated_normal(rho_hat, sigma/np.sqrt(Q))
    
    return samples

print("✓ Gibbs sampler defined")

In [ ]:
def evaluate_on_test_set(model, rho_range, sigma_range, n_test=500, device='cpu', A=5.0, T=64):
    """Evaluate model - NO TRANSFORMATIONS (same as training)"""
    model.eval()
    
    rho_true = torch.empty(n_test, device=device).uniform_(*rho_range)
    sigma_true = torch.empty(n_test, device=device).uniform_(*sigma_range)
    x_test = simulate_ar1_batch(rho_true, sigma_true, T, n_test, device=device)
    
    with torch.no_grad():
        theta_pred = model(x_test)
        # NO transformations - predict raw values just like training
        rho_pred = theta_pred[:, 0]
        sigma_pred = theta_pred[:, 1]
        
        # Optional: clip to valid ranges
        rho_pred = torch.clamp(rho_pred, -1.0, 1.0)
        sigma_pred = torch.clamp(sigma_pred, 0.0, A)
    
    error_rho = torch.abs(rho_pred - rho_true).cpu().numpy()
    error_sigma = torch.abs(sigma_pred - sigma_true).cpu().numpy()
    
    return {
        'rho_true': rho_true.cpu().numpy(),
        'sigma_true': sigma_true.cpu().numpy(),
        'rho_pred': rho_pred.cpu().numpy(),
        'sigma_pred': sigma_pred.cpu().numpy(),
        'x_test': x_test.cpu().numpy(),
        'error_rho': error_rho,
        'error_sigma': error_sigma,
        'mae_rho': error_rho.mean(),
        'mae_sigma': error_sigma.mean(),
        'mse_rho': (error_rho**2).mean(),
        'mse_sigma': (error_sigma**2).mean()
    }

# Test scenarios
test_scenarios = [
    {'name': 'Moderate', 'rho': (0.3, 0.7), 'sigma': (1.0, 3.0)},
    {'name': 'High ρ', 'rho': (0.9, 0.99), 'sigma': (0.5, 2.0)},
    {'name': 'Small σ', 'rho': (0.3, 0.7), 'sigma': (0.05, 0.3)},
    {'name': 'Both extreme', 'rho': (0.95, 0.99), 'sigma': (0.05, 0.2)},
]

results_comparison = []

for scenario in test_scenarios:
    print(f"\nEvaluating on: {scenario['name']}")
    print(f"  ρ ∈ {scenario['rho']}, σ ∈ {scenario['sigma']}")
    
    # Standard model
    results_std = evaluate_on_test_set(model_standard, scenario['rho'], scenario['sigma'], 
                                        n_test=500, device=device, A=A, T=T)
    
    # IS model
    results_is_model = evaluate_on_test_set(model_is, scenario['rho'], scenario['sigma'], 
                                             n_test=500, device=device, A=A, T=T)
    
    # Gibbs sampler
    print(f"  Running Gibbs sampler...")
    rho_gibbs_scenario = np.zeros(500)
    sigma_gibbs_scenario = np.zeros(500)
    for i in range(500):
        samples = gibbs(results_std['x_test'][i], n_iter=1000)
        rho_gibbs_scenario[i] = np.mean(samples[:, 0])
        sigma_gibbs_scenario[i] = np.mean(samples[:, 1])
    
    error_rho_gibbs = np.abs(rho_gibbs_scenario - results_std['rho_true'])
    error_sigma_gibbs = np.abs(sigma_gibbs_scenario - results_std['sigma_true'])
    mae_rho_gibbs = error_rho_gibbs.mean()
    mae_sigma_gibbs = error_sigma_gibbs.mean()
    
    print(f"  Standard - MAE(ρ): {results_std['mae_rho']:.4f}, MAE(σ): {results_std['mae_sigma']:.4f}")
    print(f"  IS       - MAE(ρ): {results_is_model['mae_rho']:.4f}, MAE(σ): {results_is_model['mae_sigma']:.4f}")
    print(f"  Gibbs    - MAE(ρ): {mae_rho_gibbs:.4f}, MAE(σ): {mae_sigma_gibbs:.4f}")
    
    improvement_rho = (results_std['mae_rho'] - results_is_model['mae_rho']) / results_std['mae_rho'] * 100
    improvement_sigma = (results_std['mae_sigma'] - results_is_model['mae_sigma']) / results_std['mae_sigma'] * 100
    
    print(f"  IS Improvement - ρ: {improvement_rho:+.1f}%, σ: {improvement_sigma:+.1f}%")
    
    results_comparison.append({
        'scenario': scenario['name'],
        'mae_rho_std': results_std['mae_rho'],
        'mae_rho_is': results_is_model['mae_rho'],
        'mae_rho_gibbs': mae_rho_gibbs,
        'mae_sigma_std': results_std['mae_sigma'],
        'mae_sigma_is': results_is_model['mae_sigma'],
        'mae_sigma_gibbs': mae_sigma_gibbs,
        'improvement_rho': improvement_rho,
        'improvement_sigma': improvement_sigma
    })

In [ ]:
# Visualize comparison: Standard NN vs IS NN vs Gibbs
df = pd.DataFrame(results_comparison)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(len(df))
width = 0.25

# MAE for rho
axes[0].bar(x - width, df['mae_rho_std'], width, label='Standard NN', alpha=0.8, edgecolor='black')
axes[0].bar(x, df['mae_rho_is'], width, label='IS NN', alpha=0.8, edgecolor='black', color='orange')
axes[0].bar(x + width, df['mae_rho_gibbs'], width, label='Gibbs', alpha=0.8, edgecolor='black', color='green')
axes[0].set_xlabel('Test Scenario')
axes[0].set_ylabel('Mean Absolute Error')
axes[0].set_title('Estimation Error: ρ')
axes[0].set_xticks(x)
axes[0].set_xticklabels(df['scenario'], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# MAE for sigma
axes[1].bar(x - width, df['mae_sigma_std'], width, label='Standard NN', alpha=0.8, edgecolor='black')
axes[1].bar(x, df['mae_sigma_is'], width, label='IS NN', alpha=0.8, edgecolor='black', color='orange')
axes[1].bar(x + width, df['mae_sigma_gibbs'], width, label='Gibbs', alpha=0.8, edgecolor='black', color='green')
axes[1].set_xlabel('Test Scenario')
axes[1].set_ylabel('Mean Absolute Error')
axes[1].set_title('Estimation Error: σ')
axes[1].set_xticks(x)
axes[1].set_xticklabels(df['scenario'], rotation=45, ha='right')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("SUMMARY OF RESULTS")
print("="*80)
print(df.to_string(index=False))

## Comprehensive Comparison: Standard NN vs IS NN vs Gibbs

In [ ]:
# Generate test dataset
n_comparison = 1000
rho_comp = torch.empty(n_comparison, device='cpu').uniform_(-1, 1)
sigma_comp = torch.empty(n_comparison, device='cpu').uniform_(0, A)
x_comp = simulate_ar1_batch(rho_comp, sigma_comp, T, n_comparison, device='cpu')

rho_true_comp = rho_comp.numpy()
sigma_true_comp = sigma_comp.numpy()
x_comp_np = x_comp.numpy()

# Get NN predictions (Standard)
model_standard.eval()
with torch.no_grad():
    x_comp_gpu = x_comp.to(device)
    theta_std = model_standard(x_comp_gpu).cpu()
    rho_std_comp = torch.clamp(theta_std[:, 0], -1.0, 1.0).numpy()
    sigma_std_comp = torch.clamp(theta_std[:, 1], 0.0, A).numpy()

# Get NN predictions (IS)
model_is.eval()
with torch.no_grad():
    theta_is = model_is(x_comp_gpu).cpu()
    rho_is_comp = torch.clamp(theta_is[:, 0], -1.0, 1.0).numpy()
    sigma_is_comp = torch.clamp(theta_is[:, 1], 0.0, A).numpy()

# Get Gibbs predictions
print("Running Gibbs sampler for comparison...")
rho_gibbs_comp = np.zeros(n_comparison)
sigma_gibbs_comp = np.zeros(n_comparison)

for i in range(n_comparison):
    if (i + 1) % 200 == 0:
        print(f"  Gibbs iteration {i+1}/{n_comparison}")
    samples = gibbs(x_comp_np[i], n_iter=1000)
    rho_gibbs_comp[i] = np.mean(samples[:, 0])
    sigma_gibbs_comp[i] = np.mean(samples[:, 1])

print("✓ All predictions obtained")

In [ ]:
# Comprehensive comparison plot: Standard NN vs IS NN vs Gibbs
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Row 1: ρ predictions
# Standard NN - rho
axes[0, 0].scatter(rho_true_comp, rho_std_comp, alpha=0.5, s=20)
axes[0, 0].plot([-1, 1], [-1, 1], 'r--', linewidth=2, label='Perfect prediction')
axes[0, 0].set_xlabel('True ρ')
axes[0, 0].set_ylabel('Predicted ρ')
axes[0, 0].set_title(f'Standard NN: ρ\nMAE={np.abs(rho_std_comp - rho_true_comp).mean():.4f}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xlim([-1, 1])
axes[0, 0].set_ylim([-1, 1])

# IS NN - rho
axes[0, 1].scatter(rho_true_comp, rho_is_comp, alpha=0.5, s=20, color='orange')
axes[0, 1].plot([-1, 1], [-1, 1], 'r--', linewidth=2, label='Perfect prediction')
axes[0, 1].set_xlabel('True ρ')
axes[0, 1].set_ylabel('Predicted ρ')
axes[0, 1].set_title(f'IS NN: ρ\nMAE={np.abs(rho_is_comp - rho_true_comp).mean():.4f}')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xlim([-1, 1])
axes[0, 1].set_ylim([-1, 1])

# Gibbs - rho
axes[0, 2].scatter(rho_true_comp, rho_gibbs_comp, alpha=0.5, s=20, color='green')
axes[0, 2].plot([-1, 1], [-1, 1], 'r--', linewidth=2, label='Perfect prediction')
axes[0, 2].set_xlabel('True ρ')
axes[0, 2].set_ylabel('Predicted ρ')
axes[0, 2].set_title(f'Gibbs: ρ\nMAE={np.abs(rho_gibbs_comp - rho_true_comp).mean():.4f}')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)
axes[0, 2].set_xlim([-1, 1])
axes[0, 2].set_ylim([-1, 1])

# Row 2: σ predictions
# Standard NN - sigma
axes[1, 0].scatter(sigma_true_comp, sigma_std_comp, alpha=0.5, s=20)
axes[1, 0].plot([0, A], [0, A], 'r--', linewidth=2, label='Perfect prediction')
axes[1, 0].set_xlabel('True σ')
axes[1, 0].set_ylabel('Predicted σ')
axes[1, 0].set_title(f'Standard NN: σ\nMAE={np.abs(sigma_std_comp - sigma_true_comp).mean():.4f}')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xlim([0, A])
axes[1, 0].set_ylim([0, A])

# IS NN - sigma
axes[1, 1].scatter(sigma_true_comp, sigma_is_comp, alpha=0.5, s=20, color='orange')
axes[1, 1].plot([0, A], [0, A], 'r--', linewidth=2, label='Perfect prediction')
axes[1, 1].set_xlabel('True σ')
axes[1, 1].set_ylabel('Predicted σ')
axes[1, 1].set_title(f'IS NN: σ\nMAE={np.abs(sigma_is_comp - sigma_true_comp).mean():.4f}')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xlim([0, A])
axes[1, 1].set_ylim([0, A])

# Gibbs - sigma
axes[1, 2].scatter(sigma_true_comp, sigma_gibbs_comp, alpha=0.5, s=20, color='green')
axes[1, 2].plot([0, A], [0, A], 'r--', linewidth=2, label='Perfect prediction')
axes[1, 2].set_xlabel('True σ')
axes[1, 2].set_ylabel('Predicted σ')
axes[1, 2].set_title(f'Gibbs: σ\nMAE={np.abs(sigma_gibbs_comp - sigma_true_comp).mean():.4f}')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)
axes[1, 2].set_xlim([0, A])
axes[1, 2].set_ylim([0, A])

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*80)
print("COMPARISON SUMMARY: Standard NN vs IS NN vs Gibbs")
print("="*80)
print(f"ρ estimation:")
print(f"  Standard NN - MAE: {np.abs(rho_std_comp - rho_true_comp).mean():.6f}")
print(f"  IS NN       - MAE: {np.abs(rho_is_comp - rho_true_comp).mean():.6f}")
print(f"  Gibbs       - MAE: {np.abs(rho_gibbs_comp - rho_true_comp).mean():.6f}")
print(f"\nσ estimation:")
print(f"  Standard NN - MAE: {np.abs(sigma_std_comp - sigma_true_comp).mean():.6f}")
print(f"  IS NN       - MAE: {np.abs(sigma_is_comp - sigma_true_comp).mean():.6f}")
print(f"  Gibbs       - MAE: {np.abs(sigma_gibbs_comp - sigma_true_comp).mean():.6f}")
print("="*80)